## Models and Training Models
Contains the code for the model and for training the model.

In [ ]:
import torch
from torch.autograd import Variable
from torch import nn
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm

### Models

In [5]:
class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels=6, kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features= 120)
        self.fc2 = nn.Linear(in_features = 120, out_features = 60)
        self.out = nn.Linear(in_features= 60, out_features = 10)

    def forward(self, tensor):
        # hidden layer 1
        tensor = self.conv1(tensor)
        tensor = F.relu(tensor)
        tensor = F.max_pool2d(tensor, kernel_size = 2, stride= 2)
        # hidden layer 2
        tensor = self.conv2(tensor)
        tensor = F.relu(tensor)
        tensor = F.max_pool2d(tensor, kernel_size = 2, stride = 2)
        # hidden layer 3
        tensor = tensor.reshape(-1, 12 * 4* 4)
        tensor = self.fc1(tensor)
        tensor = F.relu(tensor)
        # hidden layer 4
        tensor = self.fc2(tensor)
        tensor = F.relu(tensor)
        # output layer
        tensor = self.out(tensor)
        return tensor

### Code to Train the Model

In [6]:
def train_model(model, train_loader, valid_loader, optimizer, criterion, epochs):
    # Train the model
    print("Training model...")
    for e in range(epochs):
        # Set the model to training mode
        model.train()
        train_loss = 0
        # Iterate over the training data
        for images, labels in tqdm(train_loader):
            # Move the data to the device
            images, labels = images.to(device), labels.to(device)
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            output = model(images)
            loss = criterion(output, labels)
            # Backward pass
            if torch.cuda.is_available():
                with torch.cuda.amp.autocast():
                    loss.backward()
            else:
                loss.backward()
            # Update the weights
            optimizer.step()
            train_loss += loss.item()
        else:
            # Set the model to evaluation mode
            model.eval()
            valid_loss, correct = 0, 0
            total = 0
            # Iterate over the validation data
            with torch.no_grad():
                for images, labels in valid_loader:
                    images, labels = images.to(device), labels.to(device)
                    output = model(images)
                    valid_loss += criterion(output, labels).item()
                    correct += torch.sum(torch.argmax(output, dim=1) == labels).item()
                    total += len(labels)
            accuracy = correct / total
            # Save the model if the validation loss is the lowest so far
            print(f"Epoch: {e+1}/{epochs}  Training loss: {train_loss/len(train_loader):.4f}  Validation loss: {valid_loss/len(valid_loader):.4f}  Validation accuracy: {accuracy:.4f}")
    return train_loss/len(train_loader), valid_loss/len(valid_loader), accuracy